In [ ]:
import os
from timeit import default_timer as timer
import math

import numpy as np
import torch

import pytorch_kinematics as pk
from pytorch_kinematics import FKSolution 
fk = FKSolution()
from pytorch_kinematics import Jacobian 
jacobian = Jacobian()
import pytorch_seed

import pybullet as p
import pybullet_data
import time

device = "cpu"

In [4]:
urdf = "franka/fp3_franka_hand.urdf"
chain = pk.build_chain_from_urdf(open(urdf, mode="rb").read())
chain.print_tree()
print(chain.get_joint_parameter_names())
# chain = chain.to(device=device)

base
└── fp3_link0
    └── fp3_link1
        └── fp3_link2
            └── fp3_link3
                └── fp3_link4
                    └── fp3_link5
                        └── fp3_link6
                            └── fp3_link7
                                └── fp3_link8
                                    └── fp3_hand
                                        ├── fp3_hand_tcp
                                        ├── fp3_leftfinger
                                        └── fp3_rightfinger

['fp3_joint1', 'fp3_joint2', 'fp3_joint3', 'fp3_joint4', 'fp3_joint5', 'fp3_joint6', 'fp3_joint7', 'fp3_finger_joint1', 'fp3_finger_joint2']


Unknown attribute "name" in /robot[@name='fp3']/link[@name='fp3_link0']/visual[1]
Unknown attribute "name" in /robot[@name='fp3']/link[@name='fp3_link0']/collision[1]
Unknown attribute "name" in /robot[@name='fp3']/link[@name='fp3_link1']/visual[1]
Unknown attribute "name" in /robot[@name='fp3']/link[@name='fp3_link1']/collision[1]
Unknown attribute "D" in /robot[@name='fp3']/joint[@name='fp3_joint1']/dynamics
Unknown attribute "K" in /robot[@name='fp3']/joint[@name='fp3_joint1']/dynamics
Unknown attribute "mu_coulomb" in /robot[@name='fp3']/joint[@name='fp3_joint1']/dynamics
Unknown attribute "mu_viscous" in /robot[@name='fp3']/joint[@name='fp3_joint1']/dynamics
Unknown attribute "name" in /robot[@name='fp3']/link[@name='fp3_link2']/visual[1]
Unknown attribute "name" in /robot[@name='fp3']/link[@name='fp3_link2']/collision[1]
Unknown attribute "D" in /robot[@name='fp3']/joint[@name='fp3_joint2']/dynamics
Unknown attribute "K" in /robot[@name='fp3']/joint[@name='fp3_joint2']/dynamics
U

In [ ]:
serial_chain = pk.SerialChain(chain, "fp3_hand_tcp", "base")
serial_chain.print_tree()
print(serial_chain.get_joint_parameter_names())

th = torch.tensor([0.0, -math.pi / 4.0, 0.0, math.pi / 2.0, 0.0, math.pi / 4.0, 0.0], requires_grad=True)
print(len(th))

ret = fk.forward_kinematics(serial_chain,th, end_only=False)
tg = ret['fp3_hand_tcp']



base
└── fp3_link0
    └── fp3_link1
        └── fp3_link2
            └── fp3_link3
                └── fp3_link4
                    └── fp3_link5
                        └── fp3_link6
                            └── fp3_link7
                                └── fp3_link8
                                    └── fp3_hand
                                        └── fp3_hand_tcp

['fp3_joint1', 'fp3_joint2', 'fp3_joint3', 'fp3_joint4', 'fp3_joint5', 'fp3_joint6', 'fp3_joint7']
7


In [6]:
m = tg.get_matrix()
pos = m[:, :3, 3]
rot = pk.matrix_to_quaternion(m[:, :3, :3])
print("Matrix: ", m)
print("Pose: ", pos)
print("Rotation: ", rot)

Matrix:  tensor([[[-7.0711e-01, -7.0711e-01, -5.9605e-08, -4.6630e-01],
         [ 7.0711e-01, -7.0711e-01,  0.0000e+00,  0.0000e+00],
         [-4.2147e-08, -4.2147e-08,  1.0000e+00,  4.9532e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]],
       grad_fn=<ViewBackward0>)
Pose:  tensor([[-0.4663,  0.0000,  0.4953]], grad_fn=<SelectBackward0>)
Rotation:  tensor([[ 3.8268e-01, -2.7534e-08, -1.1405e-08,  9.2388e-01]],
       grad_fn=<ViewBackward0>)


In [7]:
pos.norm().backward()
print("Matrix: ", m)
print("Pose: ", pos)
print("Rotation: ", rot)

Matrix:  tensor([[[-7.0711e-01, -7.0711e-01, -5.9605e-08, -4.6630e-01],
         [ 7.0711e-01, -7.0711e-01,  0.0000e+00,  0.0000e+00],
         [-4.2147e-08, -4.2147e-08,  1.0000e+00,  4.9532e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]],
       grad_fn=<ViewBackward0>)
Pose:  tensor([[-0.4663,  0.0000,  0.4953]], grad_fn=<SelectBackward0>)
Rotation:  tensor([[ 3.8268e-01, -2.7534e-08, -1.1405e-08,  9.2388e-01]],
       grad_fn=<ViewBackward0>)


In [ ]:
serial_chain = pk.SerialChain(chain, "fp3_hand_tcp", "base")
th = torch.tensor([0.0, -math.pi / 4.0, 0.0, math.pi / 2.0, 0.0, math.pi / 4.0, 0.0])
# (1,6,7) tensor, with 7 corresponding to the DOF of the robot
J = jacobian.jacobian(serial_chain, th)

# get Jacobian in parallel and use CUDA if available
N = 1000
d = "cpu"
dtype = torch.float64

serial_chain = serial_chain.to(dtype=dtype, device=d)
# Jacobian calculation is differentiable
th = torch.rand(N, 7, dtype=dtype, device=d, requires_grad=True)
# (N,6,7)
J = jacobian.jacobian(serial_chain, th)
print("Jacobian: ",J)
# can get Jacobian at a point offset from the end effector (location is specified in EE link frame)
# by default location is at the origin of the EE frame
loc = torch.rand(N, 3, dtype=dtype, device=d)
J = jacobian.jacobian(serial_chain, th, locations=loc)

print("Jacobian: ",J)

Jacobian:  tensor([[[-2.7657e-01,  4.4431e-01, -7.8522e-02,  ..., -2.0728e-01,
           4.2249e-02,  0.0000e+00],
         [ 1.6174e-01,  3.4297e-01, -7.0815e-02,  ..., -2.0748e-02,
           6.2675e-02,  0.0000e+00],
         [-4.1633e-17, -2.9703e-01,  5.7520e-02,  ...,  4.6744e-02,
           2.1517e-01,  0.0000e+00],
         [ 4.2988e-09, -6.1105e-01,  3.7912e-01,  ...,  2.2682e-01,
           9.7089e-01, -2.1158e-01],
         [-1.5776e-08,  7.9159e-01,  2.9266e-01,  ..., -7.9054e-02,
           9.7181e-02,  7.7645e-01],
         [ 1.0000e+00, -1.5115e-08,  8.7785e-01,  ...,  9.7072e-01,
          -2.1894e-01, -5.9360e-01]],

        [[-3.9539e-01,  3.0313e-01, -1.1314e-01,  ..., -1.6867e-01,
           1.4986e-01,  0.0000e+00],
         [ 4.5900e-01,  1.5011e-01,  2.7481e-02,  ...,  6.8456e-02,
           9.1859e-02,  0.0000e+00],
         [ 0.0000e+00, -5.8679e-01,  1.2140e-01,  ...,  1.3067e-01,
           1.4531e-01,  0.0000e+00],
         [ 7.0712e-09, -4.4376e-01,  7.221

In [9]:
seed=3
max_iterations = 100
num_retries = 5
n = 100

In [10]:
pytorch_seed.seed(seed)
# device = "cuda" if torch.cuda.is_available() else "cpu"
search_path = pybullet_data.getDataPath()
chain = pk.SerialChain(chain, "fp3_hand_tcp", "base")
urdf = "franka/fp3_franka_hand.urdf"
skip = False
delay = True

chain.print_tree()

base
└── fp3_link0
    └── fp3_link1
        └── fp3_link2
            └── fp3_link3
                └── fp3_link4
                    └── fp3_link5
                        └── fp3_link6
                            └── fp3_link7
                                └── fp3_link8
                                    └── fp3_hand
                                        └── fp3_hand_tcp



'base\n└── fp3_link0\n    └── fp3_link1\n        └── fp3_link2\n            └── fp3_link3\n                └── fp3_link4\n                    └── fp3_link5\n                        └── fp3_link6\n                            └── fp3_link7\n                                └── fp3_link8\n                                    └── fp3_hand\n                                        └── fp3_hand_tcp\n'

In [11]:
  
# robot frame
pos = torch.tensor([0.0, 0.0, 0.0], device=device)
rot = torch.tensor([0.0, 0.0, 0.0, 1.0], device=device)
start_tf = pk.Transform3d(pos=pos, rot=rot, device=device)


# world frame goal
M = 1  # Only testing with a single goal
lim = torch.tensor(chain.get_joint_limits(), device=device)


# get ee pose (in robot frame)
pos = torch.tensor([0.2343, -0.5141,  0.4510], device=device)
rot = torch.tensor([0.8352,  0.0437, -0.2720, -0.4760], device=device)
end_tf = pk.Transform3d(pos=pos, rot=rot, device=device)

print("Start TF:",start_tf)
interpolated_tfs = pk.interpolate_poses(start_tf, end_tf, n)
print("Interpolated TFs:")
for tf in interpolated_tfs:
    print("\t",tf)
print("End TF:",end_tf)

all_tfs = interpolated_tfs + [end_tf]

Start TF: Transform3d(rot=tensor([[0., 0., 0., 1.]]), pos=tensor([[0., 0., 0.]]))
Interpolated TFs:
	 Transform3d(rot=tensor([[0., 0., 0., 1.]]), pos=tensor([[0., 0., 0.]]))
	 Transform3d(rot=tensor([[-1.0309e-02, -5.3940e-04,  3.3574e-03,  9.9994e-01]]), pos=tensor([[ 0.0024, -0.0052,  0.0046]]))
	 Transform3d(rot=tensor([[-0.0206, -0.0011,  0.0067,  0.9998]]), pos=tensor([[ 0.0047, -0.0104,  0.0091]]))
	 Transform3d(rot=tensor([[-0.0309, -0.0016,  0.0101,  0.9995]]), pos=tensor([[ 0.0071, -0.0156,  0.0137]]))
	 Transform3d(rot=tensor([[-0.0412, -0.0022,  0.0134,  0.9991]]), pos=tensor([[ 0.0095, -0.0208,  0.0182]]))
	 Transform3d(rot=tensor([[-0.0515, -0.0027,  0.0168,  0.9985]]), pos=tensor([[ 0.0118, -0.0260,  0.0228]]))
	 Transform3d(rot=tensor([[-0.0618, -0.0032,  0.0201,  0.9979]]), pos=tensor([[ 0.0142, -0.0312,  0.0273]]))
	 Transform3d(rot=tensor([[-0.0721, -0.0038,  0.0235,  0.9971]]), pos=tensor([[ 0.0166, -0.0364,  0.0319]]))
	 Transform3d(rot=tensor([[-0.0824, -0.0043,  0

In [12]:
# Transform to world frame for visualization
goal_tf = start_tf.compose(end_tf)
goal = goal_tf.get_matrix()
goal_pos = goal[..., :3, 3]
goal_rot = pk.matrix_to_euler_angles(goal[..., :3, :3], "XYZ")

# Initialize IK solver
ik = pk.PseudoInverseIK(chain, max_iterations=max_iterations, num_retries=num_retries,
                        joint_limits=lim.T,
                        early_stopping_any_converged=True,
                        early_stopping_no_improvement="all",
                        debug=False,
                        lr=0.2)

# Solve IK for all interpolated transforms (batch solution)
# sol = ik.parallel_solve(start_tf, end_tf, n)
sol = ik.iterative_interpolation_solve(start_tf, end_tf, n)


In [13]:

# Visualization setup
p.connect(p.GUI)
p.setRealTimeSimulation(False)
p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)
search_path = pybullet_data.getDataPath()
p.setAdditionalSearchPath(search_path)

# Setup camera view
yaw = 90
pitch = -35
dist = 1
target = np.array([0, 0, 0])
p.resetDebugVisualizerCamera(dist, yaw, pitch, target)

# Load plane
plane_id = p.loadURDF("plane.urdf", [0, 0, 0], useFixedBase=True)
p.changeVisualShape(plane_id, -1, rgbaColor=[0.3, 0.3, 0.3, 1])

# Load the robot in PyBullet
m = start_tf.get_matrix()
pos = m[0, :3, 3]
rot = m[0, :3, :3]
quat = pk.matrix_to_quaternion(rot)
pos = pos.cpu().numpy()
rot = pk.wxyz_to_xyzw(quat).cpu().numpy()


armId = p.loadURDF(urdf, basePosition=pos, baseOrientation=rot, useFixedBase=True)

# Visualize goal pose (green cone)
visId = p.createVisualShape(p.GEOM_MESH, fileName="cone.obj", meshScale=1.0,
                            rgbaColor=[0., 1., 0., 0.5])
goal_marker = p.createMultiBody(baseMass=0, baseVisualShapeIndex=visId)

# Set goal marker position and orientation
xyzw = pk.wxyz_to_xyzw(pk.matrix_to_quaternion(pk.euler_angles_to_matrix(goal_rot, "XYZ")))
p.resetBasePositionAndOrientation(goal_marker, goal_pos[0].cpu().numpy(), xyzw[0].cpu().numpy())

In [ ]:

# Original joint states
original_joint_states = []
for dof in range(p.getNumJoints(armId)):
    original_joint_states.append(p.getJointState(armId, dof)[0])  # Save the joint position
# print("Length: ",len(sol))
# Apply IK solutions to the robot's joints and visualize each
for ii in range(num_retries):

    # Reset to the original joint states
    for dof in range(len(original_joint_states)):
            p.resetJointState(armId, dof, original_joint_states[dof])
    for step in range(len(sol)):
        if delay:
            time.sleep(0.05)
        # Get IK solution for this transform
        solutions = sol[step].solutions[0,:,:]           
        # Apply the IK solution
        q = solutions[ii, :]
        for dof in range(q.shape[0]):
            p.resetJointState(armId, dof, q[dof])

        # Compute the end-effector pose
        end_effector_tf = fk.forward_kinematics(chain,q.unsqueeze(0))
        end_effector_tf = start_tf.compose(end_effector_tf)

        # Compute errors

        pos_error, rot_error = pk.compute_error(all_tfs[step], end_effector_tf)

        print(f"Step {step+1}/{len(all_tfs)} | IK solution attempt {ii+1}/{num_retries}")
        try:
            print(f"Position Error: {pos_error.item():.2f} meters")
            print(f"Rotation Error: {rot_error.item():.2f} radians")
        except:
            print(f"Position Error: {pos_error[0].item():.2f} meters")
            print(f"Rotation Error: {rot_error[0].item():.2f} radians")
        # write_to_csv(pos_error.item(), rot_error.item(), n, max_iterations, "interpolation.csv")
        # write_to_csv_with_step(pos_error.item(), rot_error.item(), n, max_iterations, step + 1, "steps.csv")
        # time.sleep(0.02)
        if skip:
            input("Press Enter to continue to the next step")
    if skip:
        input("Press Enter to continue to the next solution")
p.disconnect()

Step 1/101 | IK solution attempt 1/5
Position Error: 0.08 meters
Rotation Error: 3.60 radians
Step 2/101 | IK solution attempt 1/5
Position Error: 0.06 meters
Rotation Error: 3.49 radians
Step 3/101 | IK solution attempt 1/5
Position Error: 0.02 meters
Rotation Error: 3.15 radians
Step 4/101 | IK solution attempt 1/5
Position Error: 0.03 meters
Rotation Error: 3.13 radians
Step 5/101 | IK solution attempt 1/5
Position Error: 0.04 meters
Rotation Error: 3.12 radians
Step 6/101 | IK solution attempt 1/5
Position Error: 0.05 meters
Rotation Error: 3.11 radians
Step 7/101 | IK solution attempt 1/5
Position Error: 0.06 meters
Rotation Error: 3.10 radians
Step 8/101 | IK solution attempt 1/5
Position Error: 0.07 meters
Rotation Error: 3.09 radians
Step 9/101 | IK solution attempt 1/5
Position Error: 0.08 meters
Rotation Error: 3.08 radians
Step 10/101 | IK solution attempt 1/5
Position Error: 0.09 meters
Rotation Error: 3.08 radians
Step 11/101 | IK solution attempt 1/5
Position Error: 0.10 